In [78]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import re
import csv

In [2]:
def extract_job_title_from_result(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [3]:
def extract_company_from_result(soup):
    companies = []
    recJobLoc = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        jobrecloc = div.find_all(name="div", attrs = {"class":"recJobLoc"})
        for jobid in jobrecloc:
            recJobLoc.append(jobid.get("id").replace("recJobLoc_", ""))
        company = div.find_all(name="span", attrs = {"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs = {"class":"result - link - source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return (companies, recJobLoc)

In [4]:
def extract_location_from_result(soup):
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)

In [9]:
def extract_salary_from_result(soup):
    salaries = []
    qualificationKeywords = ['qualifications','qualify', 'required','requirements', 'nice to have', 'what we’re looking for','who we’re looking for']
    for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
        for item in div.find_all('b'):
            print("ITEM: ",item.text)
            salaries.append(item.text)
        """try:
            salaries.append(div.find("nobr").text)
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"salaryText"})
                div_three = div_two.find("div")
                salaries.append(div_three.text.strip())
            except:
                salaries.append("Nothing_found")"""
    return(salaries)

In [7]:
def extract_summary_from_result(soup):
    summaries = []
    spans = soup.findAll("span", attrs={"class": "summary"})

    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

In [48]:
titlesOfInterest = []

In [97]:
URL = "https://www.indeed.com/jobs?q=data+engineer+%2420%2C000&l=Toronto&start=10"

page = requests.get(URL)
soup = BeautifulSoup(page.text, "html.parser")

#print(extract_job_title_from_result(soup))
#print(extract_company_from_result(soup))
#print(extract_location_from_result(soup))
companyNames, indeedIdentification = extract_company_from_result(soup)
for i in range(len(companyNames)):
    cmpName = companyNames[i].replace(" ", "%20")
    cmpName = cmpName.replace("'", "%27")
    url1 = (f'https://www.indeed.com/viewjob?cmp={cmpName}&jk={indeedIdentification[i]}')
    print(url1)
    page = requests.get(url1)
    soup1 = BeautifulSoup(page.text, "html.parser")
    titlesOfInterest.append(extract_salary_from_result(soup1))
    

https://www.indeed.com/viewjob?cmp=Halcyon%20Health&jk=e92dca228cf574e8
https://www.indeed.com/viewjob?cmp=EY&jk=146a36cd2c632f99
ITEM:  Skills
ITEM:  
Qualifications
https://www.indeed.com/viewjob?cmp=Fidelity%20National%20Financial&jk=af747db88947fba8
ITEM:  Fidelity National Financial, Inc. is an equal opportunity employer. All employees are required to successfullly pass a criminal records check.
https://www.indeed.com/viewjob?cmp=BOOKBYTE&jk=0f46f8ec975b2fa1
https://www.indeed.com/viewjob?cmp=FedEx%20Services&jk=3ee72403951e66f7
ITEM:  Company:
ITEM:  Job Title:
ITEM:  Job Requisition Number:
ITEM:  Category:
ITEM:  Locations:
ITEM:  
Job Summary:
ITEM:  
We are looking for a well versed, hands on, Data Architect who has experience defining future state strategy and architecture principles for a cloud based Data Lake supporting advanced analytic solutions at enterprise scale.
ITEM:  
As a Domain Architect - Data Lake & Analytics Architect you will be responsible for designing and 

In [93]:
titlesOfInterest = [i for i in flatten(titlesOfInterest)]

In [99]:
titlesOfInterest

['JOB SUMMARY',
 'QUALIFICATIONS',
 'Education',
 'Training and Experience',
 'Health and Background Requirements',
 'COVID-19 Call Center Representative ',
 'Pay rate- $13/hr',
 'Remote - Eastern Time Zone',
 'Responsibilities: ',
 'Requirements: ',
 'Position Objective',
 'Responsibilities',
 'Qualifications',
 'Special Notes',
 'SUMMARY\n',
 'ESSENTIAL RESPONSIBILITIES\n',
 'QUALIFICATIONS\n',
 'SKILLS / COMPETENCIES\n',
 'PHYSICAL REQUIREMENTS\n',
 'COMPANY OVERVIEW',
 '\nMISSION STATEMENT',
 '\nJOB SUMMARY',
 '\nCORE JOB DUTIES',
 'REQUIRED EXPERIENCE, EDUCATION AND SKILLS',
 'ADDITIONAL REQUIREMENTS',
 'Responsibilities: ',
 'Description Detail: ',
 'Calibration, Drifts, Precision Calibrations & Method 21 Monitoring',
 'Documentation & Reporting',
 'Daily Documentation',
 'Physical Requirements ',
 'Safety ',
 'Must Be Able to Travel',
 'About EMSI',
 'United States Postal Service',
 'External Publication for Job Posting 10503639',
 '\nBranch',
 '\nJob Posting Period',
 '\nJob Ti

In [100]:
len(titlesOfInterest)

659

In [73]:
with open('qualification.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["qualifications"])
    for t in titlesOfInterest:
        writer.writerow([t])

,qualifications
0,JOB SUMMARY
1,QUALIFICATIONS
2,Education
3,Training and Experience
4,Health and Background Requirements
...,...
455,Nice to Haves:
456,What We Offer:
457,Leadership Experience
458,Technical Skills
